In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import h5py
import os
import glob
import warnings
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

import sys
basedir = '/project/vitelli/jonathan/REDO_fruitfly/'
sys.path.insert(0, os.path.join(basedir, 'src'))

warnings.filterwarnings('ignore')


## Build Composite library

In [30]:
from utils.library.composite_library import *
from utils.library.extra_functions import *
        
libraries = [
    ('m_ij', 'Public/Halo_Hetero_Twist[ey53]_Hetero/Sqh-GFP', 'tensor_library', 'tensor_SVDPipeline_PMG_CF.pkl'),
    ('v', 'Public/Halo_Hetero_Twist[ey53]_Hetero/Sqh-GFP', 'tensor_library', None), #Not focused on flow dynamics
    ('c', 'Public/WT/ECad-GFP', 'scalar_library', None), #Not focused on cadherin dynamics, just include it as a source
]
    
extra_functions = [
    lambda data: material_derivative_terms(data, key='m_ij'),
    #lambda data: add_static_sources(data, couple='m_ij'),
    lambda data: active_strain_decomposition_mesh(data, key='m_ij'),
    #lambda data: symmetric_tensor_couple(data, ['m_ij', 'm_ij']),
    lambda data: symmetric_tensor_couple(data, ['m_ij', 'E_full', 'E_passive', 'E_active']),
    lambda data: remove_terms(data, max_space_order=1),
    lambda data: multiply_tensor_by_scalar(data, None, ['c']),
]


with h5py.File(os.path.join(libraries[0][1], 'derivative_library.h5'), 'r') as h5f:
    embryoIDs = list(h5f.keys())

with h5py.File('Public/mesh_dynamics_fitting.h5', 'w') as data:
    with tqdm(total=len(embryoIDs)) as pbar:
        pbar.set_description('Collecting embryo data')
        for embryoID in embryoIDs:
            pbar.set_postfix(embryoID=embryoID)
            group, feature_names = fill_group_info(data, embryoID, libraries, filename='mesh_derivative_library.h5')
            collect_library(group, 
                            libraries, 
                            feature_names, 
                            extra_functions=extra_functions)
            take_time_derivatives(group, libraries, window_length=5)
            library_to_tangent_space(group, libraries)
            pbar.update()

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

## Learn Myosin model

In [31]:
import pickle as pk
from utils.modeling.sindy_utils import *

path = 'Public/Halo_Hetero_Twist[ey53]_Hetero/Sqh-GFP'
key = 'm_ij'

info = dict(
    key=key,
    tmin=0, tmax=20,
    threshold=1e-2,
    alpha=1e1,
    n_candidates_to_drop=0,
    n_models=10,
    subset_fraction=.25,
    overleaf_only=False,
    collect_function=collect_mesh_data,
)

with h5py.File('Public/mesh_dynamics_fitting.h5', 'r') as h5f:
    sindy = fit_sindy_model(h5f, **info)

  0%|          | 0/12 [00:00<?, ?it/s]

(125, 12663, 21) (125, 12663, 1)
D_t m_ij = 0.323 E_passive + -3.007 E_passive Tr(m_ij) + -0.362 c E_passive + 3.492 c E_passive Tr(m_ij) + -0.012 c m_ij + 0.044 c {m_ij, E_passive} + 0.036 m_ij + 0.098 {m_ij, E_passive}
